In [1]:
import math
import numpy as np
import gurobipy as GRB
import gurobipy as gp
from gurobipy import GRB,quicksum
import time
n=8
size=100
K=166

w=[7,11,15,19,23,27,35,42]
b=[52,97,140,191,85,129,26,20]
pi=[0 for i in range(K)]
I=len(w)


In [2]:
tic=time.perf_counter()
results=[]
#producing an initial feasible solution and PI
st=[0,0,0,0,0,0,0,0]
df=[]
for i in range(len(w)):
    st=[0,0,0,0,0,0,0,0]
    e=int(100/w[i])
    st[i]=e
    st2=[st for i in range(math.ceil(b[i]/e))]
    for stock in st2:
        df.append(stock)
len(df)

K = 166 #len obtained from the initial heuristically feasible solution above
f = [1 for i in range(0, K)]
pi=[0 for i in range(K)]

#attaching initial feasible solutions to F (for ys) and H (for xs)
F=[]
F.append(f)
F
H=[]
H.append(df)

UB=1000
LB=-1000
iterations=0
while abs(UB-LB)>0.001:
    #
    iterations+=1
    #SUBPROBLEM 1
    sub1=gp.Model()
    y=sub1.addVars(K,vtype=GRB.BINARY)
    sub1.setObjective(quicksum(y[k] for k in range(K))-100*quicksum(y[k]*pi[k] for k in range(K)),GRB.MINIMIZE)
    sub1.optimize()
    sub1.objVal
    
    #SUBPROBLEM 2
    sub2=gp.Model()
    x=sub2.addVars(len(w),K,vtype=GRB.INTEGER)
    objective=0
    for i in range(len(w)):
        objective+=w[i]*quicksum(pi[k]*x[i,k] for k in range(K))    
    sub2.setObjective(objective,GRB.MINIMIZE)
    for i in range(len(w)):
        sub2.addConstr(quicksum(x[i,k] for k in range(K) )==b[i])
    sub2.optimize()
    sub2.objVal
    
    #OBTAINING THE LB, ATTACHING THE GENERATED X AND Y TO THE H
    lag=sub1.objVal+sub2.objVal
    if lag>LB:
        LB=lag

    f = []
    for k in range(K):
        f.append(y[k].x)
    F.append(f)
    df = np.array([[0 for j in range(0, I)] for u in range(0, K)])
    for k in range(K):
        for i in range(I):
            df[k][i] = x[i, k].x
    H.append(df)


    #DUAL MASTER PROBLEM
    dmp=gp.Model()
    theta1=dmp.addVar(vtype=GRB.CONTINUOUS,name='theta1') #determines whether stock k is used or not
    theta2=dmp.addVars(len(w),vtype=GRB.CONTINUOUS,name='theta2') #determines whether stock k is used or not

    pi=dmp.addVars(K,vtype=GRB.CONTINUOUS, name='pi')

    dmp.setObjective(theta1+quicksum(theta2[i] for i in range(len(w))) ,GRB.MAXIMIZE)
    for f in F:
        dmp.addConstr(theta1<=quicksum(f[k] for k in range(K))-100*quicksum(f[k]*pi[k] for k in range(K)))

    for h in H:
        for i in range(len(w)):
            dmp.addConstr(theta2[i]<=w[i]*quicksum(h[k][i]*pi[k] for k in range(K) ))

    dmp.optimize()

    UB=dmp.objVal

    #UPDATING THE PI
    pij=[]
    for i in range(len(pi)):
        pij.append(pi[i].x)
    #pi=[pi.x for i in range(pi)]
    pi=pij
    print(dmp.objval)
    results.append((sub1.objVal,sub2.objVal,dmp.objVal))
    





toc=time.perf_counter()
print('LowerBound' , LB)
print('Runtime ' ,toc-tic)
print('No of iterations', iterations)

Academic license - for non-commercial use only - expires 2021-06-23
Using license file C:\Users\OIBRA\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0xcb19c01c
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8

  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 286.9600000
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 45 rows, 175 columns and 425 nonzeros
Model fingerprint: 0xc749dfd0
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 2 rows and 148 columns
Presolve time: 0.00s
Presolved: 43 rows, 2

       0    2.8725496e+02   6.782821e+01   0.000000e+00      0s
      25    1.5477111e+02   0.000000e+00   0.000000e+00      0s

Solved in 25 iterations and 0.01 seconds
Optimal objective  1.547711111e+02
154.77111111111117
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0xcad75041
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [7e-01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -154.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -154 
No other solutions better than -154

Optimal solution found (tolerance 1.00e-04)
Best objective -1.540000000000e+02, best bound -1.540000000000e+02, gap 0.0000%
G

Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0x4fb5b5b7
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 204.9714286
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 108 rows, 175 columns and 628 nonzeros
Model fingerprint: 0xa5a

Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 2 rows and 100 columns
Presolve time: 0.00s
Presolved: 133 rows, 75 columns, 521 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8725496e+02   1.262029e+02   0.000000e+00      0s
      35    1.5158146e+02   0.000000e+00   0.000000e+00      0s

Solved in 35 iterations and 0.01 seconds
Optimal objective  1.515814615e+02
151.5814615384615
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0x232116f2
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [8e-02, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heur

...

Model fingerprint: 0xdb357cce
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 935 rows, 174 columns, 9853 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8725496e+02   8.468490e+02   0.000000e+00      0s
     545    1.4368218e+02   0.000000e+00   0.000000e+00      0s

Solved in 545 iterations and 0.03 seconds
Optimal objective  1.436821779e+02
143.68217787485665
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0x34baf1af
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e-03, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range 

  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -1.5025332

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -1.50253 
No other solutions better than -1.50253

Optimal solution found (tolerance 1.00e-04)
Best objective -1.502533151028e+00, best bound -1.502533151028e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0x15d4b6df
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 143.6437795
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iter

Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 64.4867 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.448671522790e+01, best bound 6.448671522790e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 999 rows, 175 columns and 11003 nonzeros
Model fingerprint: 0xe88c94be
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 998 rows, 174 columns, 10892 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8725496e+02   9.019616e+02   0.000000e+00      0s
     455    1.4348000e+02   0.000000e+00   0.000000e+00      0s

Solved in 455 ite

Optimal objective  1.434800000e+02
143.48
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0x1b0373af
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [3e-04, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -2.1698446

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -2.16984 
No other solutions better than -2.16984

Optimal solution found (tolerance 1.00e-04)
Best objective -2.169844627015e+00, best bound -2.169844627015e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns a

Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0xdd203893
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 143.4800000
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 95.0885 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.508851165153e+01, best bound 9.508851165153e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1062 rows, 175 columns and 11397 nonzeros
Model fingerprint: 0x6a23d340
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Obj

In [39]:
print('theta1',theta1.X)
dmp.printAttr('X')


theta1 0.0

    Variable            X 
-------------------------
   theta2[0]         3.64 
   theta2[1]        10.67 
   theta2[2]           21 
   theta2[3]        36.29 
   theta2[4]        19.55 
   theta2[5]        34.83 
   theta2[6]          9.1 
   theta2[7]          8.4 
       pi[0]         0.01 
       pi[1]         0.01 
       pi[2]         0.01 
       pi[3]         0.01 
       pi[4]         0.01 
       pi[5]         0.01 
       pi[6]         0.01 
       pi[7]         0.01 
       pi[8]         0.01 
       pi[9]         0.01 
      pi[10]         0.01 
      pi[11]         0.01 
      pi[12]         0.01 
      pi[13]         0.01 
      pi[14]         0.01 
      pi[15]         0.01 
      pi[16]         0.01 
      pi[17]         0.01 
      pi[18]         0.01 
      pi[19]         0.01 
      pi[20]         0.01 
      pi[21]         0.01 
      pi[22]         0.01 
      pi[23]         0.01 
      pi[24]         0.01 
      pi[25]         0.01 
      pi[26]     

In [30]:
print('LowerBound' , LB)
print('Runtime ' ,toc-tic)
print('No of iterations', iterations)

LowerBound 143.48
Runtime  313.07899069999985
No of iterations 119


In [4]:
x

{(0, 0): <gurobi.Var C0 (value 52.0)>,
 (0, 1): <gurobi.Var C1 (value -0.0)>,
 (0, 2): <gurobi.Var C2 (value -0.0)>,
 (0, 3): <gurobi.Var C3 (value -0.0)>,
 (0, 4): <gurobi.Var C4 (value -0.0)>,
 (0, 5): <gurobi.Var C5 (value -0.0)>,
 (0, 6): <gurobi.Var C6 (value -0.0)>,
 (0, 7): <gurobi.Var C7 (value -0.0)>,
 (0, 8): <gurobi.Var C8 (value -0.0)>,
 (0, 9): <gurobi.Var C9 (value -0.0)>,
 (0, 10): <gurobi.Var C10 (value -0.0)>,
 (0, 11): <gurobi.Var C11 (value -0.0)>,
 (0, 12): <gurobi.Var C12 (value -0.0)>,
 (0, 13): <gurobi.Var C13 (value -0.0)>,
 (0, 14): <gurobi.Var C14 (value -0.0)>,
 (0, 15): <gurobi.Var C15 (value -0.0)>,
 (0, 16): <gurobi.Var C16 (value -0.0)>,
 (0, 17): <gurobi.Var C17 (value -0.0)>,
 (0, 18): <gurobi.Var C18 (value -0.0)>,
 (0, 19): <gurobi.Var C19 (value -0.0)>,
 (0, 20): <gurobi.Var C20 (value -0.0)>,
 (0, 21): <gurobi.Var C21 (value -0.0)>,
 (0, 22): <gurobi.Var C22 (value -0.0)>,
 (0, 23): <gurobi.Var C23 (value -0.0)>,
 (0, 24): <gurobi.Var C24 (value -0.

In [32]:
tee

[5]